### __Group Project - Predicting Airbnb Listing Prices in Melbourne, Australia__
---

## Data Cleaning, Missing Observations and Feature Engineering

#### Structured Function-Based Data Preprocessing

Instead of writing multiple code lines to clean data, I decided to use a more structured and systematic approach by creating functions for each step of data preprocessing. This approach is expected to provide more critical advantages for a comprehensive machine learning pipeline development.

**1. Prevents Data Leakage**.
All preprocessing parameters, especially for data imputation are computed exclusively from training data and applied consistently to test data. This ensures no future information leaks into model training.

**2. Modular and Maintainable**.
Each preprocessing step is encapsulated in a function with single responsibility. This makes debugging easier, enables targeted modifications, and improves code readability for team collaboration.

**3. Reproducible and Scalable**.
The structured pipeline can be systematically applied to new datasets, unit tested independently, and easily extended with additional preprocessing steps as needed.

**4. Quality Control**.
Function-based validation at each step enables early error detection and ensures consistent transformations across all data splits, leading to more reliable model performance.

---

To begin, we start by import necessary libraries and load the datasets to start data preparation process:

In [13]:
import pandas as pd
import numpy as np
import json
import re

from math import sqrt, sin, cos, asin, radians
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# --- Data Loading ---
def load_dataset(path: str) -> pd.DataFrame:
    return pd.read_csv(path)

**Approach 1: Numerical Feature Cleaning**

• **Converting percentage columns**: Removing '%' symbols from `host_response_rate` and `host_acceptance_rate`, converting to decimal format (e.g., "85%" → 0.85).

• **Cleaning price data**: Removing ' and ',' symbols from `price` column to extract numerical values (e.g., "$1,250" → 1250.0).

• **Processing bathroom text**: Converting mixed text in `bathrooms` column by replacing "half" with "0.5" and extracting numerical values using regex.

• **Standardizing data types**: Converting all cleaned values to float type for consistent numerical format.

In [15]:
# --- Feature Cleaning ---
def clean_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean numerical features by extracting numerical values and removing text.
    Specifically handles 'host_response_rate', 'host_acceptance_rate', 'price',
    and 'bathrooms' columns to convert them from mixed text/numeric format to pure numeric values.
    """
    percent_cols = ['host_response_rate', 'host_acceptance_rate']
    for col in percent_cols:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = df[col].str.replace('%', '').astype(float) / 100

    if 'price' in df.columns and df['price'].dtype == 'object':
       df['price'] = df['price'].str.replace('[\$,]', '', regex=True).astype(float)

    if 'bathrooms' in df.columns and df['bathrooms'].dtype == 'object':
        df['bathrooms'] = df['bathrooms'].str.replace('half', '0.5', case=False)
        df['bathrooms'] = df['bathrooms'].str.extract(r'(\d+\.?\d*)').astype(float)
    return df

**Approach 2: Feature Engineering from Multi-Information Columns**

Given: Existing features containing multiple items of information are host_verifications, and amentities so new features will be created from these two.

- **Feature 1 - `verification_count`**: Counting the number of host verification methods from `host_verifications` column by splitting comma-separated values.

- **Feature 2 - `amenity_count`**: Converting `amenities` column into total count of available amenities by splitting the comma-separated list.

- **Features 3-9 - Luxury amenity indicators**: Creating seven binary features (0/1) for specific amenities:
  - `has_pool`: Pool or hot tub availability
  - `has_heating_cooling`: Air conditioning or heating systems
  - `has_coffee_maker`: Coffee-making equipment
  - `has_parking`: Any type of parking availability
  - `has_private_balcony`: Private outdoor space
  - `has_high_quality_bedding`: Premium bedding amenities
  - `has_scenic_view`: Properties with views  

- **Feature 10 - `luxury_amenity_count`**: Summing all luxury amenity indicators to create a composite luxury score.

In [16]:
# --- New Feature Creating ---
def create_new_features(df):
    """
    Create new features from existing multi-information columns (host_verifications and amenities).
    Extracts counts and key indicators to convert complex text data into usable numerical features.
    """
    # Feature 1: Count of host verification methods
    if 'host_verifications' in df.columns:
        df['verification_count'] = df['host_verifications'].apply(lambda x: len(x.split(',')) if x else 0)

    if 'amenities' in df.columns:
        # Feature 2: Total amenity count
        df['amenity_count'] = df['amenities'].str.split(', ').apply(len)

        # Feature 3-9: Has luxury amenities and scenic view
        luxury_keywords = {
            'has_pool': ['pool', 'hot tub'],
            'has_heating_cooling': ['air conditioning', 'ac', 'heating', 'central heating'],
            'has_coffee_maker': ['coffee maker', 'nespresso', 'espresso machine'],
            'has_parking': ['free parking', 'paid parking', 'garage', 'carport', 'street parking'],
            'has_private_balcony': ['private patio or balcony', 'balcony', 'terrace'],
            'has_high_quality_bedding': ['high quality linens', 'extra pillows and blankets',
                                         'room-darkening shades', 'blackout curtains',
                                         'comfortable bedding', 'premium linens'],
            'has_scenic_view': ['view']  # generic catch-all for anything containing 'view'
        }

        for col, keywords in luxury_keywords.items():
            pattern = r'|'.join(rf'\b{re.escape(kw)}\b' for kw in keywords)
            df[col] = df['amenities'].str.contains(pattern, case=False, regex=True).astype(int)

        # Feature 4: Count of luxury amenities
        df['luxury_amenity_count'] = df[list(luxury_keywords.keys())].sum(axis=1)

    return df

**Approach 3: Missing Value Imputation**

- **Text columns imputation**: Filling missing values in descriptive text columns (`name`, `description`, `neighborhood_overview`, `host_about`) with default text "no information" to maintain data completeness.

- **Categorical columns imputation**: Using mode (most frequent value) from training data to fill missing values in categorical columns like `host_response_time`, `host_is_superhost`, `property_type`, and `room_type`.

- **Numerical columns imputation**: Filling missing values in numerical columns (review scores, rates, counts) with mean values calculated from the training dataset to preserve statistical properties.

- **Date-related columns imputation**: For `days_since_first_review` and `days_since_last_review`, using maximum values to handle properties without review history by assuming they are new listings.

- **Training-based imputation**: Using statistics from training data (mode/mean) for both training and test datasets to prevent data leakage and maintain consistency across datasets.

In [17]:
# --- Missing Value Imputation ---
def impute_missing(df: pd.DataFrame, train_df: pd.DataFrame, fill_text='no information') -> pd.DataFrame:
    """
    Generalized missing value imputation based on inferred data types and characteristics.
    - Text columns: filled with constant text
    - Categorical columns: filled with mode from training
    - Numeric columns (generalized): filled with mean from training
    - Date-like columns: filled with max from test
    """
    # Explicit imputation for known text columns
    text_cols = ['name', 'description', 'neighborhood_overview', 'host_about']
    for col in [c for c in text_cols if c in df.columns]:
        df[col].fillna(fill_text, inplace=True)

    # Explicit imputation for known categorical columns
    cat_cols = ['host_response_time', 'host_is_superhost', 'host_location',
                'host_neighbourhood', 'property_type', 'room_type', 'neighbourhood_cleansed']
    for col in [c for c in cat_cols if c in df.columns]:
        df[col].fillna(train_df[col].mode()[0], inplace=True)

    # Generalized numeric + date-like handling
    for col in df.columns:
        if df[col].isnull().sum() == 0:
            continue # Skip columns with no missing values

        if pd.api.types.is_numeric_dtype(df[col]):
            # If column name suggests it's date-related but stored as numeric, use max value (e.g. "days_since_*")
            if 'days_since' in col or 'date' in col:
                df[col].fillna(df[col].max(), inplace=True)

            # Otherwise, use mean from training set; fallback to current df mean if missing
            else:
                df[col].fillna(train_df[col].mean(), inplace=True)

        elif pd.api.types.is_datetime64_any_dtype(df[col]):
            df[col].fillna(df[col].max(), inplace=True)


    return df

**Approach 4: Categorical Variable Encoding**

##### 4.1 Amenities Encoding
- **JSON parsing**: Converting amenities from JSON-like strings into lists for processing.
- **Top-5 selection**: Identifying the 5 most frequent amenities from training data to create binary features.
- **Binary encoding**: Creating `has_[amenity_name]` features (0/1) for each top-5 amenity.
- **Other category**: Adding `has_other_amenities` binary feature for properties with amenities outside the top-5.

##### 4.2 Neighbourhood Encoding
- **Multi-criteria selection**: Choosing neighbourhoods based on three criteria:
  - Top 10 most frequent neighbourhoods
  - Top 10 highest-priced neighbourhoods  
  - Top 10 lowest-priced neighbourhoods
- **Binary encoding**: Creating `neigh_[neighbourhood_name]` binary features for selected neighbourhoods.
- **Implicit other handling**: Neighbourhoods not selected are implicitly grouped as "other" (all binary features = 0).

##### 4.3 Other Features Encoding
- **Ordinal encoding**: Converting `host_response_time` to numerical scale (within an hour=4, few hours=3, within a day=2, few days or more=1).
- **Boolean conversion**: Mapping 't'/'f' values to 1/0 for columns like `host_is_superhost`, `instant_bookable`.
- **One-hot encoding**: Creating dummy variables for `bath_type` and `room_type`.
- **Top-5 + Other**: For `property_type`, limiting to 5 most frequent categories from training data, grouping remaining as "Other", then one-hot encoding.

##### Key Principles Applied
- **Training-based statistics**: Using training data frequencies to determine top categories for consistent encoding.
- **Handling rare categories**: Grouping infrequent values into "Other" category as specified.
- **Consistent encoding**: Applying same encoding rules to both training and test datasets.

In [18]:
# --- 4.1 Amenities Encoding ---
def encode_amenities(df: pd.DataFrame, train_df: pd.DataFrame) -> pd.DataFrame:
    """
    Create binary features for the top 5 most common amenities from training data.
    Parses amenities from JSON-like strings, identifies the most frequent amenities,
    and creates indicator variables for popular amenities plus a catch-all for other amenities.
    """
    def parse(amenities):
        try:
            if isinstance(amenities, str):
                return json.loads(amenities.replace("'", '"'))
            return []
        except:
            return []

    if 'amenities' in df.columns:
        # Parse amenities for both dataframes
        df['amenities_list'] = df['amenities'].apply(parse)
        train_df['amenities_list'] = train_df['amenities'].apply(parse)

        # Get top 5 amenities from training data
        all_items = [item for sublist in train_df['amenities_list'] for item in sublist]
        top_5 = pd.Series(all_items).value_counts().nlargest(5).index.tolist()

        # Create binary features for top 5 amenities
        for amenity in top_5:
            key = f'has_{amenity.lower().replace(" ", "_")}'
            df[key] = df['amenities_list'].apply(lambda x: int(amenity in x))

        # Create feature for other amenities
        df['has_other_amenities'] = df['amenities_list'].apply(lambda x: int(any(i not in top_5 for i in x)))

    return df

# --- 4.2 Location Encoding ---
def encode_neighbourhoods(df: pd.DataFrame, train_df: pd.DataFrame) -> pd.DataFrame:
    if 'neighbourhood_cleansed' not in df.columns:
        return df
    top_freq = train_df['neighbourhood_cleansed'].value_counts().nlargest(10).index.tolist()
    top_price = train_df.groupby('neighbourhood_cleansed')['price'].mean().nlargest(10).index.tolist()
    low_price = train_df.groupby('neighbourhood_cleansed')['price'].mean().nsmallest(10).index.tolist()
    selected = list(set(top_freq + top_price + low_price))

    for neigh in selected:
        df[f"neigh_{neigh.replace(' ', '_').lower()}"] = (df['neighbourhood_cleansed'] == neigh).astype(int)
    return df

# --- 4.3 Other Features Encoding ---
def encode_other_features(df: pd.DataFrame, train_df: pd.DataFrame) -> pd.DataFrame:
    """
    Encode features respectively:
    - Ordinal encode host_response_time
    - Convert booleans ('t'/'f') to 1/0
    - One-hot encode room_type and bath_type
    - One-hot encode top-5 property_type (others grouped)
    """
    # Ordinal encode response time
    if 'host_response_time' in df.columns:
        response_order = {'within an hour': 4, 'within a few hours': 3, 'within a day': 2, 'a few days or more': 1}
        df['host_response_time'] = df['host_response_time'].map(response_order).fillna(0).astype(int)

    # One-hot encode bath_type and room_type
    for col in ['bath_type', 'room_type']:
        if col in df.columns:
            df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)

    # Property type: Top-5 + Other one-hot
    if 'property_type' in df.columns:
        top5 = train_df['property_type'].value_counts().nlargest(5).index
        df['property_type_limited'] = df['property_type'].where(df['property_type'].isin(top5), 'Other')
        df = pd.concat([df, pd.get_dummies(df['property_type_limited'], prefix='property_type')], axis=1)

    # Convert boolean columns
    bool_cols = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']
                # this doesn't include 'has_availabity' b/c this column contains only 1 value 't', which will not meaningful to the machine learning process
    for col in bool_cols:
        if col in df.columns:
            df[col] = df[col].map({'t': 1, 'f': 0}).fillna(-1)

    # Final Safe-Net: turn all new introduced bool dtype columns to int
        bool_dtype_cols = df.select_dtypes(include='bool').columns
        df[bool_dtype_cols] = df[bool_dtype_cols].astype(int)


    return df

**Approach 5: Additional Data Preparation Steps**

To enhance model performance and capture complex data patterns, the following additional preprocessing steps were applied:
1. *Additional Feature Engineering (Sections 5.1-5.3):*
    - Applied before imputation (Q3) to ensure any new missing values introduced can be immediately handled.
    
        ---
2. *Sentiment Analysis (Section 5.4)*
    - Performed after handling missing values (Q3), focusing on text features.
        
        ---
3. *Feature Transformation (Section 5.5)*
    - Applied to reduce feature bias and improve numerical stability
    
        ---
4. *Pipeline Integration and Execution (Sections 5.6–5.7)*
   - Ensures a logically structured and reproducible processing flow.  
   - Facilitates debugging, testing, and future enhancements to the preprocessing pipeline.

**5.1 Extra Feature Engineering**

**Goal:** Transform raw textual and date fields into structured, informative features.

- Bathroom and Room Type Extraction
  - `bath_type`: Categorizes bathrooms as 'Shared', 'Private', or 'Normal' based on keywords.
  - `room_type`: Standardizes room type entries using keyword matching (e.g., 'entire', 'shared').
      
      ---
        
- Date-Based Features
  - Converts `host_since`, `first_review`, and `last_review` to datetime.
  - Derived features:
    - `host_experience_days`: Days since host joined.
    - `days_since_first_review`: Days since the first review.
    - `days_since_last_review`: Days since the last review.
    
    ---
        
- Popularity Metrics
  - `reviews_per_day`: Measures listing popularity as number_of_reviews ÷ host_experience_days.
        
      ---
        
- Temporal Scaling
  - `review_recency_score`: Recency scaled with exponential decay to favor recent reviews.
  - `host_experience_maturity`: Log-transformed host experience for diminishing returns effect.

In [19]:
# --- 5.1 Creating Extra Features ---
def add_extra_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create additional engineered features from existing data including bathroom/room type extraction,
    date-based features for host experience and review timing, popularity metrics, and temporal scaling.
    Transforms raw data into more meaningful features for machine learning models.
    """
    # --- Bath & Room Type Extraction ---
    bath_mapping = {'hared': 'Shared', 'rivate': 'Private'}
    room_mapping = {'entire': 'Entire', 'shared': 'Shared', 'private': 'Private', 'hotel': 'Hotel'}

    if 'bathrooms' in df.columns:
        df['bath_type'] = df['bathrooms'].astype(str).apply(
            lambda x: next((v for k, v in bath_mapping.items() if k in x), 'Normal')
        )
        df['bathroom_count'] = df['bathrooms']

    if 'room_type' in df.columns:
        df['room_type'] = df['room_type'].astype(str).str.lower().apply(
            lambda x: next((v for k, v in room_mapping.items() if k in x), 'Unknown')
        )

    # --- Date Parsing ---
    date_cols = ['host_since', 'first_review', 'last_review']
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # --- Experience & Review Age Features ---
    ref_date = max([df[col].max() for col in date_cols if col in df.columns])
    feature_ages = {
        'host_since': 'host_experience_days',
        'first_review': 'days_since_first_review',
        'last_review': 'days_since_last_review'
    }
    for col, new_col in feature_ages.items():
        if col in df.columns:
            df[new_col] = (ref_date - df[col]).dt.days

    df.drop([col for col in date_cols if col in df.columns], axis=1, inplace=True)

    # --- Popularity Signal ---
    if {'number_of_reviews', 'host_experience_days'}.issubset(df.columns):
        df['reviews_per_day'] = df['number_of_reviews'] / df['host_experience_days'].replace(0, np.nan)

    # --- Temporal Scaling ---
    if 'days_since_last_review' in df.columns:
        df['review_recency_score'] = np.exp(-df['days_since_last_review'] / 365)

    if 'host_experience_days' in df.columns:
        df['host_experience_maturity'] = np.log1p(df['host_experience_days'])

    return df

**5.2 Distance-Based Features**

**Goal:** Quantify geographic desirability based on proximity to Melbourne landmarks.

- Distance Calculations (Haversine Formula)
  - `dist_to_cbd`: Distance to Melbourne CBD.
  - `dist_to_beach`: Distance to the nearest beach.
  - `dist_to_mall`: Distance to the nearest shopping mall.
  
  ---

- Proximity Count
  - `count_nearby_sights`: Number of attractions (CBD, malls, beaches) within a 0.8 km radius (equivalent to 10-minute walking distance, a favourable distance for AirBnb tenants).

In [20]:
# --- 5.2 Creating Distance Features ---
def add_distance_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate location-based features using haversine distance formula.
    Computes distances to Melbourne CBD, nearest beach, nearest shopping mall,
    and counts nearby attractions within 0.8km radius to capture location desirability.
    """
    def haversine(lon1, lat1, lon2, lat2):
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
        dlon, dlat = lon2 - lon1, lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        return 6371 * 2 * asin(sqrt(a))

    # Central Business District (CBD) coordinates
    cbd = (-37.8136, 144.9631)

    # List of major Melbourne beaches
    beaches = [
        (-37.847039, 144.945525),  # Port Melbourne
        (-37.854066, 144.960893),  # South Melbourne / Middle Park
        (-37.864000, 144.982000),  # St Kilda
        (-37.892639, 144.988190),  # Elwood
        (-37.905000, 144.996000),  # Brighton (Dendy St)
        (-37.937000, 145.009000),  # Hampton
        (-37.952500, 145.012311),  # Sandringham
        (-37.989640, 145.058720),  # Mentone
        (-38.000000, 145.083330),  # Mordialloc
        (-38.042559, 145.109469),  # Edithvale
        (-38.053029, 145.113617),  # Chelsea
        (-38.083330, 145.133330),  # Carrum
        (-38.100000, 145.133330),  # Seaford
        (-38.140343, 145.119280),  # Frankston
        (-37.870911, 144.830323),  # Altona
        (-37.863930, 144.894320)   # Williamstown
    ]

    # List of popular Melbourne shopping malls
    malls = [
        (-37.887085, 145.081528),  # Chadstone Shopping Centre
        (-37.770700, 144.884800),  # Highpoint Shopping Centre
        (-37.958000, 145.050000),  # Westfield Southland
        (-37.783333, 145.121667),  # Westfield Doncaster
        (-37.868970, 145.241480),  # Westfield Knox
        (-38.017833, 145.302500),  # Westfield Fountain Gate
        (-37.813060, 145.229170),  # Eastland Shopping Centre
        (-37.738330, 145.029720),  # Northland Shopping Centre
        (-37.876390, 145.165638),  # The Glen (Glen Waverley)
        (-37.812400, 144.963800),  # Emporium Melbourne
        (-37.810272, 144.962646),  # Melbourne Central
        (-37.810696, 144.965716),  # QV Melbourne
        (-37.813002, 144.938031),  # The District Docklands
        (-37.875213, 144.679659),  # Pacific Werribee
        (-37.652004, 145.023085),  # Pacific Epping
        (-37.819560, 145.121410),  # Box Hill Central (South)
        (-37.812724, 145.010977),  # Victoria Gardens (Richmond)
        (-37.825196, 144.949879),  # DFO South Wharf
        (-37.733125, 144.906547),  # DFO Essendon
        (-37.971261, 145.088120),  # DFO Moorabbin
        (-37.651367, 145.071752),  # Westfield Plenty Valley
        (-37.699266, 144.775813),  # Watergardens Town Centre
        (-37.992893, 145.173138),  # Parkmore Shopping Centre
        (-38.141951, 145.124068)   # Bayside Shopping Centre
    ]

    df['dist_to_cbd'] = df.apply(lambda r: haversine(r['longitude'], r['latitude'], cbd[1], cbd[0]), axis=1)
    df['dist_to_beach'] = df.apply(lambda r: min(haversine(r['longitude'], r['latitude'], lon, lat) for lat, lon in beaches), axis=1)
    df['dist_to_mall'] = df.apply(lambda r: min(haversine(r['longitude'], r['latitude'], lon, lat) for lat, lon in malls), axis=1)

    top_sights = [cbd] + beaches + malls
    def count_nearby_sights(row):
        return sum(
            1 for lat, lon in top_sights
            if haversine(row['longitude'], row['latitude'], lon, lat) <= 0.8
        )

    df['count_nearby_sights'] = df.apply(count_nearby_sights, axis=1)

    return df

**5.3 Interaction Features**

**Goal:** Capture important combinations of features that express non-linear relationships.

- Superhost Experience
  - `superhost_experience`: Interaction of superhost flag and host experience.
  
    ---
- Space Efficiency
  - `crowding_index`: Accommodates ÷ (bedrooms + bathrooms) for space utilization.
  
    ---
- Luxury and More Capacity Impact
  - `luxury_entire_home`: Product of luxury amenity count and entire-home flag.
  
    ---
- Location-View with Good Rating
  - `cbd_view_interaction`: Combines `dist_to_cbd` with view indicator or value rating.
  
    ---
- Rating-Volume Interaction
  - `rating_review_volume`: Review score multiplied by reviews per day.

In [21]:
# --- 5.3 Creating Interaction Features ---
def add_interaction_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create interaction features by combining existing features to capture non-linear relationships.
    These features represent meaningful combinations like experienced superhosts, space efficiency,
    luxury entire homes, location-view interactions, and rating-popularity synergies.
    """
    if 'is_superhost' in df.columns and 'host_experience_days' in df.columns:
        df['superhost_experience'] = df['is_superhost'] * df['host_experience_days']

    if 'accommodates' in df.columns and 'bedrooms' in df.columns and 'bathroom_count' in df.columns:
        df['crowding_index'] = df['accommodates'] / (df['bedrooms'] + df['bathroom_count']).replace(0, np.nan)

    if 'amenity_count' in df.columns and 'room_type_Entire' in df.columns:
        df['luxury_entire_home'] = df['luxury_amenity_count'] * df['room_type_Entire']

    if 'dist_to_cbd' in df.columns and 'has_scenic_view' in df.columns:
        df['cbd_view_interaction'] = df['dist_to_cbd'] * df['has_scenic_view']

    if 'dist_to_cbd' in df.columns and 'review_scores_value' in df.columns:
        df['cbd_view_interaction'] = df['dist_to_cbd'] * df['review_scores_value']

    if 'review_scores_rating' in df.columns and 'reviews_per_day' in df.columns:
        df['rating_review_volume'] = df['review_scores_rating'] * df['reviews_per_day']

    return df

**5.4 Sentiment Analysis**

**Goal:** Extract emotional tone from descriptive text fields.

- **VADER Sentiment Scores**
  - Applied to `neighborhood_overview` and `host_about`.
  - Outputs:
    - `neighborhood_overview_sentiment`
    - `host_about_sentiment`
  - Compound scores (range -1 to 1) indicate overall sentiment polarity.

In [22]:
# --- 5.4 Creating Sentiment Features ---
def add_sentiment_scores(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean text and apply VADER sentiment analysis to specific columns:
    'description', 'neighborhood_overview', and 'host_about'.

    Cleaning includes:
    - Removing HTML tags
    - Removing non-alphanumeric characters (except basic punctuation)
    - Trimming extra whitespace

    Add sentiment analysis scores to text columns using VADER sentiment analyzer.
    Analyzes neighborhood_overview and host_about text to extract sentiment polarity scores,
    which can indicate positive or negative sentiment that may influence booking decisions.
    """
    sid = SentimentIntensityAnalyzer()

    def clean_text(text):
        if not isinstance(text, str):
            return ""
        text = re.sub(r'<[^>]+>', ' ', text)  # Remove HTML tags
        text = re.sub(r'[^\w\s.,!?\'-]', '', text)  # Remove unwanted non-alphanumerics
        text = re.sub(r'\s+', ' ', text).strip()  # Normalize whitespace
        return text

    columns = ['neighborhood_overview', 'host_about']
    for col in columns:
        if col in df.columns:
            df[col] = df[col].apply(clean_text)
            df[f'{col}_sentiment'] = df[col].apply(
                lambda x: sid.polarity_scores(x)['compound'] if x else 0.0
            )

    return df

**5.5 Mathematical Feature Transformations**

**Goal:** Normalize skewed distributions and better capture non-linear relationships.

- **Log1p Transform (Skew Reduction)**
  - Applied to variables like `minimum_nights`, `availability_365`, and date differences.
  
    ---

- **Square Root Transform (Diminishing Returns)**
  - Used on count features such as `accommodates`, `bedrooms`, `amenity_count`.
  
    ---

- **Square Transform (Quadratic Relationships)**
  - Applied to distance-based features (`dist_to_cbd`, etc.) to capture non-linear distance effects.

In [23]:
# --- 5.5 Feature Transfromations ---
def apply_feature_transformations(df: pd.DataFrame) -> pd.DataFrame:
    """
    Apply mathematical transformations to features to improve model performance.
    Uses log1p for skewed variables, sqrt for diminishing returns features,
    and squares for distance features to better capture non-linear relationships.
    """
    # Reduce skew: apply log1p
    log1p_cols = [
        'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_maximum_nights',
        'availability_365', 'host_experience_days', 'days_since_first_review', 'days_since_last_review',
        'reviews_per_month', 'dist_to_beach', 'dist_to_mall', 'host_experience_days'
    ]
    for col in log1p_cols:
        if col in df.columns:
            df[f'{col}_log1p'] = np.log1p(df[col].clip(lower=0))

    # Diminishing returns: sqrt
    diminishing_cols = ['accommodates', 'bedrooms', 'bathroom_count', 'amenity_count', 'count_nearby_sights']
    for col in diminishing_cols:
        if col in df.columns:
            df[f'{col}_sqrt'] = np.sqrt(df[col].clip(lower=0))

    # Location non-linearity: square
    distance_cols = ['dist_to_cbd', 'dist_to_beach', 'dist_to_mall']
    for col in distance_cols:
        if col in df.columns:
            df[f'{col}_sq'] = df[col] ** 2

    return df

**5.6-5.7 Pipeline Integration**

**Goal:** Maintain consistency, avoid leakage, and ensure reproducibility.

- Steps are **sequentially applied** in the correct logical order:
  1. Feature extraction (bath/date-based)
  2. Distance and interaction features
  3. Missing value imputation
  4. Categorical encoding
  5. Sentiment scoring
  6. Final transformations
  
  ---

- Ensures **identical preprocessing** for both training and test sets using training-derived stats.
- Verifies column alignment and handles column drops/exports systematically.

In [24]:
# --- 5.6 Cleanup & Export ---
def finalize_and_export(df: pd.DataFrame, output_path: str):
    """
    Removes unnecessary columns from the DataFrame and exports the cleaned dataset to CSV.
    """
    drop_cols = [
        'source', 'name', 'host_name', 'description', 'neighborhood_overview', 'property_type_limited',
        'host_about', 'host_location',  'host_neighbourhood', 'neighbourhood', 'host_verifications', 'bathrooms',
        'amenities', 'amenities_list', 'property_type', 'bath_type', 'room_type', 'has_availability', 'neighbourhood_cleansed'
    ]
    df = df.drop(columns=[col for col in drop_cols if col in df.columns], errors='ignore')
    df.to_csv(output_path, index=False)
    return df

# --- 5.7 Run Pipeline ---
def preprocess(df: pd.DataFrame, ref_df: pd.DataFrame) -> pd.DataFrame:
    """
    Applies all preprocessing steps to transform raw data into model-ready features.
    """
    df = clean_features(df) # Approach 1
    df = create_new_features(df) # Approach 2

    df = add_extra_features(df) # Approach 5.1 --- add extra features ---
    df = add_distance_features(df) # Approach 5.2 --- add distance features ---
    df = add_interaction_features(df) # Approach 5.3 --- add interaction features ---

    df = impute_missing(df, ref_df) # Approach 3

    df = encode_amenities(df, ref_df) # Approach 4
    df = encode_neighbourhoods(df, ref_df) # Approach 4
    df = encode_other_features(df, ref_df) # Approach 4

    #df = add_sentiment_scores(df) # Approach 5.4 --- sentiment analysis ---
    df = apply_feature_transformations(df) # Q5.5 --- feature transformation ---

    return df

def run_pipeline():
    """
    Loads datasets, applies preprocessing pipeline, and exports cleaned data.
    """
    train_df = load_dataset('train.csv')
    test_df = load_dataset('test.csv')

    train_df = preprocess(train_df, train_df)
    test_df = preprocess(test_df, train_df)

    finalize_and_export(train_df, 'train_cleaned.csv')
    finalize_and_export(test_df, 'test_cleaned.csv')

run_pipeline() # Complete preprocessing data 

C:\Users\Trung\AppData\Local\Temp\ipykernel_5988\1040662810.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_sqrt'] = np.sqrt(df[col].clip(lower=0))
C:\Users\Trung\AppData\Local\Temp\ipykernel_5988\1040662810.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_sq'] = df[col] ** 2
C:\Users\Trung\AppData\Local\Temp\ipykernel_5988\1040662810.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi